In [1]:
from tf_version import SSD300v2 as tf_ssd
from torch_version import SSD300v2 as torch_ssd
from torchsummary import summary

import torch
import tensorflow as tf
import numpy as np
tf.__version__

2021-08-29 03:19:02.279761: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


'2.3.0'

In [2]:
torch_model = torch_ssd((300, 300, 3), num_classes=21)
torch_model.float()
torch_model.eval()
with torch.no_grad():
    summary(torch_model.to('cuda'), (3, 300, 300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 300, 300]           1,792
            Conv2d-2         [-1, 64, 300, 300]          36,928
         MaxPool2d-3         [-1, 64, 150, 150]               0
            Conv2d-4        [-1, 128, 150, 150]          73,856
            Conv2d-5        [-1, 128, 150, 150]         147,584
         MaxPool2d-6          [-1, 128, 75, 75]               0
            Conv2d-7          [-1, 256, 75, 75]         295,168
            Conv2d-8          [-1, 256, 75, 75]         590,080
            Conv2d-9          [-1, 256, 75, 75]         590,080
        ZeroPad2d-10          [-1, 256, 76, 76]               0
        MaxPool2d-11          [-1, 256, 38, 38]               0
           Conv2d-12          [-1, 512, 38, 38]       1,180,160
           Conv2d-13          [-1, 512, 38, 38]       2,359,808
           Conv2d-14          [-1, 512,

/home/thea/2021/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [3]:
tf_model = tf_ssd((300, 300, 3), num_classes=21)
tf_model.summary()

2021-08-29 03:19:16.140168: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-08-29 03:19:16.140282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-29 03:19:16.140662: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1070 computeCapability: 6.1
coreClock: 1.7465GHz coreCount: 15 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 238.66GiB/s
2021-08-29 03:19:16.140676: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-08-29 03:19:16.141841: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-08-29 03:19:16.142861: I tensorflow/stream_executor/platform/


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 300, 300, 64) 1792        input_1[0][0]            

In [4]:
all_names = [name for name, par in torch_model.named_parameters()]
all_names = set(all_names)

In [5]:
fst_step = []

with torch.no_grad():
    torch_model.cpu()
    for name, par in torch_model.named_parameters():
        ss = name.split('.')
        layer_name, par_subname = ss[0], ss[1]
        
        #print(layer_name, par_subname)
        
        if par_subname in ['weight', 'bias'] and ('conv' in layer_name or 'fc' in layer_name):
            fst_step += [name]
            print(layer_name, par_subname)
            
            weights = tf_model.get_layer(layer_name).weights
            
            if par_subname == 'weight':
                weight = weights[0].numpy().transpose([3, 2, 0, 1])
                par[:, :, :, :] = torch.tensor(weight)
            else:
                bias = weights[1].numpy()
                par[:] = torch.tensor(bias)
                
        elif par_subname in ['weight', 'bias']:
            fst_step += [name]
            print(layer_name, par_subname)
            
            weights = tf_model.get_layer(layer_name).weights
            
            if par_subname == 'weight':
                weight = weights[0].numpy().transpose([1, 0])
                par[:, :] = torch.tensor(weight)
            else:
                bias = weights[1].numpy()
                par[:] = torch.tensor(bias)
        elif par_subname == 'conv4_3_norm_gamma':
            fst_step += [name]
            print(layer_name, par_subname)
            
            weights = tf_model.get_layer(layer_name).weights
            
            weight = weights[0].numpy().reshape(1, 512, 1, 1)
            par[:, :, :, :] = torch.tensor(weight)
        else:
            raise Exception('what????')
            
fst_step = set(fst_step)

conv1_1 weight
conv1_1 bias
conv1_2 weight
conv1_2 bias
conv2_1 weight
conv2_1 bias
conv2_2 weight
conv2_2 bias
conv3_1 weight
conv3_1 bias
conv3_2 weight
conv3_2 bias
conv3_3 weight
conv3_3 bias
conv4_1 weight
conv4_1 bias
conv4_2 weight
conv4_2 bias
conv4_3 weight
conv4_3 bias
conv5_1 weight
conv5_1 bias
conv5_2 weight
conv5_2 bias
conv5_3 weight
conv5_3 bias
fc6 weight
fc6 bias
fc7 weight
fc7 bias
conv6_1 weight
conv6_1 bias
conv6_2 weight
conv6_2 bias
conv7_1 weight
conv7_1 bias
conv7_2 weight
conv7_2 bias
conv8_1 weight
conv8_1 bias
conv8_2 weight
conv8_2 bias
conv4_3_norm conv4_3_norm_gamma
conv4_3_norm_mbox_loc weight
conv4_3_norm_mbox_loc bias
conv4_3_norm_mbox_conf weight
conv4_3_norm_mbox_conf bias
fc7_mbox_conf weight
fc7_mbox_conf bias
fc7_mbox_loc weight
fc7_mbox_loc bias
conv6_2_mbox_conf weight
conv6_2_mbox_conf bias
conv6_2_mbox_loc weight
conv6_2_mbox_loc bias
conv7_2_mbox_conf weight
conv7_2_mbox_conf bias
conv7_2_mbox_loc weight
conv7_2_mbox_loc bias
conv8_2_mbox_con

In [6]:
all_names - fst_step

set()

In [7]:
torch.save(torch_model.state_dict(), 'SSD300v2_torch.pth')

In [8]:
frame = (np.random.randn(300, 300, 3) * 255).astype(np.float32)
frame_torch = np.copy(frame.transpose([2, 0, 1]))

In [9]:
with torch.no_grad():
    torch_model.eval()
    torch_model.float()
    torch_out = torch_model(torch.tensor(frame_torch)[None]).numpy()

In [11]:
tf_out = tf_model.predict(frame[None])[1]

2021-08-29 03:19:31.406660: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-08-29 03:19:31.508763: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


In [12]:
tf_out.shape, tf_out

((1, 7308, 33),
 array([[[ 1.0973424 ,  0.03411341, -0.77243614, ...,  0.1       ,
           0.2       ,  0.2       ],
         [-0.5467808 , -1.0718251 ,  0.26539892, ...,  0.1       ,
           0.2       ,  0.2       ],
         [-1.1499668 , -0.03241672,  0.8324232 , ...,  0.1       ,
           0.2       ,  0.2       ],
         ...,
         [ 0.00574203, -0.03945673, -0.01702614, ...,  0.1       ,
           0.2       ,  0.2       ],
         [ 0.05235558,  0.00177961, -0.02288734, ...,  0.1       ,
           0.2       ,  0.2       ],
         [ 0.01338555,  0.02763657,  0.0285685 , ...,  0.1       ,
           0.2       ,  0.2       ]]], dtype=float32))

In [13]:
torch_out.shape

(1, 7308, 33)